# DS20: Richter's Predictor: Modeling Earthquake Damage

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# Wrangle Data

## Import Data

In [17]:
dtypes = {'geo_level_1_id': 'object'}#,
          #'geo_level_2_id': 'object',
          #'geo_level_3_id': 'object'}

df = pd.merge(pd.read_csv('data/train_values.csv', dtype=dtypes),
              pd.read_csv('data/train_labels.csv')).set_index('building_id')

## EDA

- One possible problem: `NaN`'s masquerading as `0`s.

In [18]:
df.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3


In [19]:
df.select_dtypes('object').nunique()

geo_level_1_id            31
land_surface_condition     3
foundation_type            5
roof_type                  3
ground_floor_type          5
other_floor_type           4
position                   4
plan_configuration        10
legal_ownership_status     4
dtype: int64

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   geo_level_1_id                          260601 non-null  object
 1   geo_level_2_id                          260601 non-null  object
 2   geo_level_3_id                          260601 non-null  object
 3   count_floors_pre_eq                     260601 non-null  int64 
 4   age                                     260601 non-null  int64 
 5   area_percentage                         260601 non-null  int64 
 6   height_percentage                       260601 non-null  int64 
 7   land_surface_condition                  260601 non-null  object
 8   foundation_type                         260601 non-null  object
 9   roof_type                               260601 non-null  object
 10  ground_floor_type                       260601 non-

# Split Data

In [21]:
target = 'damage_grade'
X = df.drop(columns=target)
y = df[target]

In [23]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Establish Baseline

In [25]:
print('Baseline Accuracy:', y_train.value_counts(normalize=True).max())

Baseline Accuracy: 0.5697045280122793


# Build Model

In [31]:
model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    RandomForestClassifier(random_state=42, n_jobs=-1)
)

In [32]:
model.fit(X_train, y_train);

/Users/nicholascifuentes-goodbody/.local/share/virtualenvs/DS20-richter-GFwViywz/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


# Check Metrics

Accuracy

In [40]:
print('Training Accuracy:', accuracy_score(y_train, model.predict(X_train)))
print('Validation Accuracy:', accuracy_score(y_val, model.predict(X_val)))

Training Accuracy: 0.9867613200306984
Validation Accuracy: 0.7172348957234128


In [37]:
print('Training F1:', f1_score(y_train, model.predict(X_train), average='micro'))
print('Validation F1:', f1_score(y_val, model.predict(X_val), average='micro'))

Training F1: 0.9867613200306984
Validation F1: 0.7172348957234128


# Make Submission

In [41]:
X_test = pd.read_csv('data/test_values.csv', dtype=dtypes, index_col='building_id')
X_test.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0


In [42]:
y_pred = model.predict(X_test)

In [44]:
y_pred[:10]

array([3, 2, 2, 1, 3, 2, 1, 3, 2, 2])

In [46]:
submission = pd.DataFrame({'damage_grade':y_pred}, index=X_test.index)
submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3


In [54]:
pd.Timestamp.now().strftime('%Y-%m-%d_%H-%M')

'2020-10-09_15-35'

In [47]:


submission.to_csv('submissions/2020-10-09_predictions.csv', index=True)